In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models as models
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms as T
from PIL import Image
import os
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,f1_score

In [ ]:
image_size = 128
transform = T.Compose([
    T.Resize((128,128)),
    T.ToTensor()
])
batch_size = 64
num_epochs = 5
folders = ['Normal','COVID','Lung_Opacity','Viral Pneumonia']
num_classes = len(folders)

In [ ]:
class DatasetFromImage(Dataset):
    def __init__(self,image_folder,transform,train):
        self.image_folder = image_folder
        self.transform = transform
        self.images = self.fetch_train_images() if train else self.fetch_test_images()
        
    def __getitem__(self,index):
        img_path = self.images[index]
        a = img_path.split('-')
        img = Image.open(os.path.join(self.image_folder,a[0],self.images[index])).convert('RGB')
        img = self.transform(img)
        label = folders.index(a[0])
        return img,label
    
    def fetch_train_images(self):
        images = []
        for i,x in enumerate(folders):
            for j,y in enumerate(os.listdir(os.path.join(self.image_folder,x))):
                if(j<1000):
                    images.append(y)
                else:
                    break
        return images
    
    def fetch_test_images(self):
        images = []
        for i,x in enumerate(folders):
            for j,y in enumerate(os.listdir(os.path.join(self.image_folder,x))):
                if(j>1000 and j<1300):
                    images.append(y)
                elif(j>1300):
                    break
        return images
    
    def __len__(self):
        return len(self.images)

In [ ]:
train_set = DatasetFromImage('../input/covid19-radiography-database/COVID-19_Radiography_Dataset',transform,True)
train_dl = DataLoader(train_set,batch_size,shuffle=True)
test_set = DatasetFromImage('../input/covid19-radiography-database/COVID-19_Radiography_Dataset',transform,False)
test_dl = DataLoader(test_set,256,shuffle=True)

In [ ]:
for image,label in train_set:
    plt.imshow(image.permute(1,2,0))
    break

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class ConvNet(nn.Module):
  def __init__(self):
    # super here used access method of parent class
    # dont worry much just boiler plate
    super(ConvNet,self).__init__()
    # In conv layer in_channels== input; out_channels=output; kernel_size=filter size
    self.conv1=nn.Conv2d(in_channels=3,out_channels=8,kernel_size=3)
    # Linear layer in_features is input, how 8*31*31 came is explained in above comment
    # out_features= output
    self.fc1=nn.Linear(in_features=8*31*31,out_features=32)
    self.out=nn.Linear(in_features=32,out_features=2)

  def forward(self,l):
    # this method implements forward propagation
    # So, layers are structured as such

    # 1 Conv layer
    # may be thinking self.conv1 is an layer object instance how can we call as if it a function
    # Checkout python documents __call__ this special method is used, so that instances behaves like function
    # __call__ this special method invokes anytime the object instance is called. This interacts with forward method.
    l=self.conv1(l)
    l=F.relu(l)
    l=F.max_pool2d(l,kernel_size=2)

    # linear and final layer
    # -1 indicates, give any number of batch size
    l=l.reshape(-1,8*31*31)
    l=self.fc1(l)
    l=self.out(l)

    return l

In [ ]:
opt = optim.Adam(model.parameters(),lr=0.0001)
criterion = nn.CrossEntropyLoss()

In [ ]:
def acc():
    for image,label in test_dl:
        image,label = image.to(device),label.to(device)
        out = model(image)
        _,out = torch.max(out,1)
        return (torch.sum((label==out))/len(label)).item()
acc()

In [ ]:
losses = []
accuracies = []

In [ ]:
for epoch in range(num_epochs):
    for i,(image,label) in enumerate(train_dl):
        image,label = image.to(device),label.to(device)
        out = model(image)
        loss = criterion(out,label)
        opt.zero_grad()
        loss.backward()
        opt.step()
        if(i%5==0):
            accuracy = acc()
            losses.append(loss.item())
            accuracies.append(accuracy)
            print(epoch,i,loss.item(),accuracy)

In [ ]:
plt.plot(losses)
plt.title('Loss vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('loss')

In [ ]:
plt.plot(accuracies)
plt.title('Accuracy vs Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

In [ ]:
def con_mat():
    for image,label in test_dl:
        image,label = image.to(device),label.to(device)
        out = model(image)
        _,out = torch.max(out,1)
        cm = confusion_matrix(out.cpu().detach().numpy().round(),label.cpu().numpy())
        disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=folders)
        disp.plot()
        break
con_mat()

In [ ]:
def calc_f1_score():
    for image,label in test_dl:
        torch.cuda.empty_cache()
        image,label = image.to(device),label.to(device)
        out = model(image)
        _,out = torch.max(out,1)
        return f1_score(out.cpu().detach().numpy(),label.cpu().numpy(),average='macro')
calc_f1_score()